# Project Euler Problem #8

Find the greatest product of K consecutive digits in the N digit number.

Input Format 
First line contains T that denotes the number of test cases. 
First line of each test case will contain two integers N & K.
Second line of each test case will contain a N digit integer. 

Output Format 
Print the required answer for each test case.

Constraints 
1≤T≤100 
1≤K≤7 
K≤N≤1000

##Let's take a sample solution to this problem

In [1]:
def largest_consecutive_product(n, k):
    product_list = []
    tmp = 1
    start_idx = 0
    end_idx = k
    for number in n:
        if len(n) - start_idx < k:
            break
        for num in n[start_idx:end_idx]:
            tmp *= int(num)
        product_list.append(tmp)
        tmp = 1
        start_idx += 1
        end_idx += 1
    return max(product_list)

In [39]:
import random
number_to_test = ''.join([str(random.randint(0,10)) for _ in range(1000)])
largest_consecutive_product(number_to_test, 25)

125406234494160076800

Let's review the solution above and make changes one by one to it to make it more readable and more Pythonic.

Notice that this solution maintains temporary variables start_idx and end_idx. These are used to keep track of our location in the array, and we manually increment them. Python provides a handy enumerate function to solve this problem automatically.

The above code could more concisely be written like this:

In [42]:
def largest_consecutive_product(n, k):
    product_list = []
    tmp = 1
    start_idx = 0
    end_idx = k
    # I've added the enumerate function to count up the index automatically.
    for start_idx, number in enumerate(n):
        print(start_idx)
        if len(n) - start_idx < k:
            break
        for num in n[start_idx:end_idx]:
            tmp *= int(num)
        product_list.append(tmp)
        tmp = 1
        #start_idx += 1  We can comment this out since enumerate takes care of incrementing automatically
        end_idx += 1
    return max(product_list)

Next let's get rid of the end_idx variable. Afterall, it will always be equal to start_idx + k. Let's not trobule the programmer with remembering how the end_idx variable is initialized and figuring how what it does

In [47]:
def largest_consecutive_product(n, k):
    product_list = []
    tmp = 1
    start_idx = 0
    # end_idx = k  We will no longer use this variable
    for start_idx, number in enumerate(n):
        if len(n) - start_idx < k:
            break
        #for num in n[start_idx:end_idx]: Let's replace end_idx with its logical equivalent, start_idx + k
        for num in n[start_idx:start_idx + k]:
            tmp *= int(num)
        product_list.append(tmp)
        tmp = 1
        # end_idx += 1  We no longer need to increment the variable since we deleted it
    return max(product_list)

Now let's handle the biggest problem with this code. Currently, memory is O(N). This means that as n increases by 1, memory usage in the system increases proportionally. The reason for this property is because every time we loop through an addition digit of n, we append a product to product_list. product_list grows in size as the program runs, gradually increasing the program's memory footprint until it ends. We can solve this problem by only storing the largest product at any given time.

In [50]:
def largest_consecutive_product(n, k):
    # product_list = []  We are no longer going to need to store every product
    max_product = 0     # Let's instead store a single product here
    tmp = 1
    start_idx = 0
    for start_idx, number in enumerate(n):
        if len(n) - start_idx < k:
            break
        for num in n[start_idx:start_idx + k]:
            tmp *= int(num)
        # This line ensures that if a calculated product ever exceeds max_product, then that calculated product becomes
        # the new max_product
        if tmp > max_product:
            max_product = tmp
        # product_list.append(tmp) We don't need to hang onto these anymore because of the above line
        tmp = 1
    # return max(product_list) 
    # max() would normally need to iterate over all of product_list to find its max value. 
    # Not only that, this cost is not constant but actually increases linearly with the size of the inputted n.
    # This means that we incidentally made our faster as well as more memory efficieny.
    return max_product

Let's now focus on making the code a little more conscice. I see two easy wins. First off, the inner loop can be collapased into a reduce function.

In [60]:
from functools import reduce
import operator

def largest_consecutive_product(n, k):
    max_product = 0
    # tmp = 1  This variable actually doesn't need to be initialized at all
    start_idx = 0
    for start_idx, number in enumerate(n):
        if len(n) - start_idx < k:
            break
        tmp = reduce(operator.mul, map(int, n[start_idx:start_idx + k]))
        if tmp > max_product:
            max_product = tmp
        # tmp = 1 And we also don't need to reset it to one every iteration either since it is never multiplied by
        #         itself using a *= operation anymore.
    return max_product

What the heck did I just do???
        
    tmp = reduce(operator.mul, map(int, n[start_idx:start_idx + k]))
    
The map part of the function executes first and says turn every string chatacter of the slice of n into an intger. Then the reduce function executes, multiplying ever integer digit returned from the map function by eachother.

Aside from the various PEP8 errors, this code is starting to look better. We could get rid of the if/break statements and subsume them into a for loop over a more precise range, but I want to consider a sightly different problem. We've ignore the fact that this code can be made A LOT faster. There is an algorithmically more efficieny way to solve the problem.

Currently, our solution is O(K * (N - K)). I'll go over how we derive this function in a moment. For now, note that as N increases, the runtime of the program increases. Similarly, as K increases, at laest to a certain point, the runtime of the program increases. Let's model how K affects the runtime of the program for a constant N of 1000.